# MCMC tutorial publication results and plotting
Part of the Bayesian neural networks via MCMC: a Python-based tutorial

This notebook contains the code used to generate the results and plots for the paper.

### Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import os
import numpy as np
import pandas as pd
import xarray as xr
import glob
from ipywidgets import interact, fixed, widgets

os.chdir('/project')

from publication_results.models.mcmc import MCMC
from publication_results.convergence.convergence import gelman_rubin

## Load the results from the MCMC runs
Load in the results stored in the netcdf files.

In [6]:
models = ['linear'] #,'bnn'
data = ['Sunspot', 'Abalone', 'Iris', 'Ionosphere']
results_dir = os.path.join('publication_results','results')

results = {mm: {dd: None for dd in data} for mm in models}
for this_model in models:
    mod_results_dir = os.path.join(results_dir,'{}_model'.format(this_model))
    for this_data in data:
        results_files = glob.glob(os.path.join(mod_results_dir,'mcmc_{}_*.nc'.format(this_data)))
        print('Found {} results files for {} model on {} data'.format(len(results_files),this_model,this_data))
        tmp_data = xr.open_mfdataset(results_files, combine='nested',concat_dim='chain')
        # re-ID the chains
        tmp_data.coords['chain'] = np.arange(len(results_files))
        results[this_model][this_data] = tmp_data

Found 5 results files for linear model on Sunspot data
Found 5 results files for linear model on Abalone data
Found 5 results files for linear model on Iris data
Found 5 results files for linear model on Ionosphere data


In [7]:
# load the data
x_train = {}
y_train = {}
x_test = {}
y_test = {}

for this_data in data:
    train_data   = np.loadtxt("data/{}/train.txt".format(this_data))
    test_data    = np.loadtxt("data/{}/test.txt".format(this_data))

    x_train[this_data] = train_data[:,:-1]
    y_train[this_data] = train_data[:,-1]
    x_test[this_data] = test_data[:,:-1]
    y_test[this_data] = test_data[:,-1]

# little lookup table
data_case = {'Sunspot':'regression','Abalone':'regression','Iris':'classification','Ionosphere':'classification'}

## Evaluate the model performance
Calculate the model performance metrics for the MCMC samples.

In [9]:
results_table = []# pd.DataFrame(columns=['train','test','model','data'])
for this_model in models:
    for this_data in data:
        train_dim = results[this_model][this_data].coords['train_idx'].shape[0]
        test_dim = results[this_model][this_data].coords['test_idx'].shape[0]
        train_pred = results[this_model][this_data].train_pred.values.reshape((-1,train_dim))
        test_pred = results[this_model][this_data].test_pred.values.reshape((-1,test_dim))
        
        # Print the train/test RMSE or accuracy in tabular format
        if data_case[this_data] == 'regression':
            train_perf = np.array([MCMC.rmse(train_pred[_,:], y_train[this_data]) for _ in np.arange(train_pred.shape[0])])
            test_perf = np.array([MCMC.rmse(test_pred[_,:], y_test[this_data]) for _ in np.arange(test_pred.shape[0])])

        elif data_case[this_data] == 'classification':
            train_perf = np.array([MCMC.accuracy(train_pred[_,:], y_train[this_data]) for _ in np.arange(train_pred.shape[0])])
            test_perf = np.array([MCMC.accuracy(test_pred[_,:], y_test[this_data]) for _ in np.arange(test_pred.shape[0])])

        results_table.append(
            {
                'train_mean':train_perf.mean(),'test_mean':test_perf.mean(),
                'train_std':train_perf.std(),'test_std':test_perf.std(),
                'model':this_model,'data':this_data
            }
        )
results_table = pd.DataFrame(results_table)
results_table

,train_mean,test_mean,train_std,test_std,model,data
0,0.025536,0.021937,0.013265,0.011644,linear,Sunspot
1,0.086159,0.086281,0.007322,0.007165,linear,Abalone
2,89.541917,84.248143,3.681478,6.331859,linear,Iris
3,88.739853,85.393912,1.455964,2.266621,linear,Ionosphere


In [36]:
# Run the convergence diagnostics
for this_model in models:
    for this_data in data:
        print('Convergence diagnostics for {} model on {} data'.format(this_model,this_data))
        # combine the parameters together into a single xarray
        params = results[this_model][this_data].drop(['train_pred','test_pred','train_sim','test_sim','rmse'])
        params = xr.concat(
            params,
            dim='param'
        )
        hh = 1



Convergence diagnostics for linear model on Sunspot data


TypeError: can only concatenate xarray Dataset and DataArray objects, got <class 'str'>

In [37]:
params

<xarray.Dataset>
Dimensions:    (samples: 7500, train_idx: 298, test_idx: 198, chain: 5)
Coordinates:
  * samples    (samples) int32 0 1 2 3 4 5 6 ... 7494 7495 7496 7497 7498 7499
  * train_idx  (train_idx) int32 0 1 2 3 4 5 6 7 ... 291 292 293 294 295 296 297
  * test_idx   (test_idx) int32 0 1 2 3 4 5 6 7 ... 191 192 193 194 195 196 197
  * chain      (chain) int64 0 1 2 3 4
Data variables:
    w0         (chain, samples) float64 dask.array<chunksize=(1, 7500), meta=np.ndarray>
    w1         (chain, samples) float64 dask.array<chunksize=(1, 7500), meta=np.ndarray>
    w2         (chain, samples) float64 dask.array<chunksize=(1, 7500), meta=np.ndarray>
    w3         (chain, samples) float64 dask.array<chunksize=(1, 7500), meta=np.ndarray>
    b          (chain, samples) float64 dask.array<chunksize=(1, 7500), meta=np.ndarray>
    tau        (chain, samples) float64 dask.array<chunksize=(1, 7500), meta=np.ndarray>
    rmse       (chain, samples) float64 dask.array<chunksize=(1, 7500), meta=np.ndarray>

In [34]:
tmp_data

<xarray.Dataset>
Dimensions:     (chain: 5, samples: 5000, train_idx: 245, test_idx: 106)
Coordinates:
  * samples     (samples) int32 0 1 2 3 4 5 6 ... 4994 4995 4996 4997 4998 4999
  * train_idx   (train_idx) int32 0 1 2 3 4 5 6 ... 238 239 240 241 242 243 244
  * test_idx    (test_idx) int32 0 1 2 3 4 5 6 7 ... 99 100 101 102 103 104 105
  * chain       (chain) int64 0 1 2 3 4
Data variables: (12/377)
    train_pred  (chain, samples, train_idx) float64 dask.array<chunksize=(1, 5000, 245), meta=np.ndarray>
    train_sim   (chain, samples, train_idx) float64 dask.array<chunksize=(1, 5000, 245), meta=np.ndarray>
    test_pred   (chain, samples, test_idx) float64 dask.array<chunksize=(1, 5000, 106), meta=np.ndarray>
    test_sim    (chain, samples, test_idx) float64 dask.array<chunksize=(1, 5000, 106), meta=np.ndarray>
    w0          (chain, samples) float64 dask.array<chunksize=(1, 5000), meta=np.ndarray>
    w1          (chain, samples) float64 dask.array<chunksize=(1, 5000), meta=np.ndarray>
    ...          ...
    w367        (chain, samples) float64 dask.array<chunksize=(1, 5000), meta=np.ndarray>
    w368        (chain, samples) float64 dask.array<chunksize=(1, 5000), meta=np.ndarray>
    w369        (chain, samples) float64 dask.array<chunksize=(1, 5000), meta=np.ndarray>
    b0          (chain, samples) float64 dask.array<chunksize=(1, 5000), meta=np.ndarray>
    b1          (chain, samples) float64 dask.array<chunksize=(1, 5000), meta=np.ndarray>
    tau         (chain, samples) float64 dask.array<chunksize=(1, 5000), meta=np.ndarray>